### Authoritarian V2

Non-graphical to provide more transparency and flexibility.
https://github.com/langchain-ai/langchain/blob/master/cookbook/multiagent_authoritarian.ipynb

In [58]:
from dotenv import load_dotenv

load_dotenv()

True

In [59]:
import functools
import random
from typing import Callable, List
# import tenacity # retrying library

from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatOpenAI

In [60]:
class WorkerAgent:
    """
    Worker agent class defining agents that can send and receive messages.
    """
    
    def __init__(self, position: str, role: str, function: str, model: ChatOpenAI) -> None:
        self.position = position
        self.role = role
        self.function = function
        self.system_message = SystemMessage(
            content=f"""You are an expert {self.position} with the role: {self.role} and function: {self.function}.
            Speak critically from the perspective of an expert: {self.position} with the role: {self.role} and function: {self.function}.
            """
            )
        self.model = model
        self.reset()

    def reset(self):
        self.recent_message = ""

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content=self.recent_message),
            ]
        )
        return message.content

    def receive(self, message: str) -> None:
        """
        Concatenates {message} spoken by {position} into message history
        """
        self.recent_message = message


class OptimisationSimulator:
    """
    Simulates a conversation between multiple agents
    """

    def __init__(self, agents: List[WorkerAgent], selection_function: Callable[[int, List[WorkerAgent]], int]) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_worker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def start(self):
        """
        Initiates the conversation with the base prompt
        """
        prompt = self.agents[0].prompt
        for agent in self.agents:
            agent.receive(prompt)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # Choose the next worker
        worker_idx = self.select_next_worker(self._step, self.agents)
        worker = self.agents[worker_idx]

        # Next worker sends message
        message = worker.send()

        # Everyone receives message
        for receiver in self.agents:
            receiver.receive(message)

        self._step += 1

        return worker.position, message

In [61]:
class WorkerIndex(BaseModel):
    """Index of the worker to select next"""
    index: int = Field(..., description="Index of the worker selected to provide feedback next")

class StopBoolean(BaseModel):
    """Boolean to stop the optimisation"""
    stop: bool = Field(..., description="Boolean value indicating whether to stop the optimisation or not.")
    
class Workforce(BaseModel):
    """Details of workforce generated by the leader agent."""
    positions: List[str] = Field(..., description="Positions of the workers in the workforce")
    roles: List[str] = Field(..., description="Roles of the workers in the workforce")
    functions: List[str] = Field(..., description="Functions of the workers in the workforce")

class Prompt(BaseModel):
    """Optimised prompt"""
    prompt: str = Field(..., description="The optimised prompt")

class DirectorAgent(WorkerAgent):
    """
    Authoritative agent that builds the workforce and directs the conversation between agents
    """

    def __init__(
            self,
            base_prompt: str,
            model: ChatOpenAI,
            stopping_probability: float,
            position: str = "Director",
            role: str = "Oversee the optimisation of prompts",
            function: str = "Interact with workers to make improvements to optimise a prompt, where a prompt is defined as 'text input to an LLM that instructs the model to perform a task'.",
            workers: List[WorkerAgent] = None,
            additional_info: str = None
            ) -> None:
        super().__init__(position, role, function, model)
        self.prompt = base_prompt
        self.position = position
        self.roles = role
        self.function = function
        self.additional_info = additional_info
        self.workers = self._generate_workforce() if workers is None else workers
        self.next_worker = ""

        self.stop = False
        self.stopping_probability = stopping_probability
        self.termination_clause = "Finish the discussion by making final carefully considered improvements to the prompt and return the optimised prompt."
        self.continuation_clause = "Do not end the discussion. Make the improvements you have carefully considered to the prompt and return the optimised prompt."

    def _generate_workforce(self) -> List[WorkerAgent]:
        """
        Generates a workforce to help optimise prompts.
        """
        workforce = []
        template = """Generate a highly tailored workforce to advise on optimising the prompt: {prompt}.
        The workforce should consist of non-leadership positions.
        State the positions, roles, and functions of the workers.
        Position is defined as the title of the worker, role is defined as the responsibility of the worker, and function is defined as the purpose of the worker.
        Carefully consider your choice of positions to ensure the workforce expertise cover the entire domain of the prompt task and can provide the most comprehensive feedback. 
        Please provide detailed descriptions of the roles and function that emphasize their skills and responsibilities. 
        If any additional information has been provided, use it to help define the workforce: {additional_info}

        Your output should be in a dictionary: {{"positions": ["position1", "position2", ...], "roles": ["role1", "role2", ...], "functions": ["function1", "function2", ...]}}.

        {format_instructions}

        Do nothing else.
        """
        pydantic_parser = PydanticOutputParser(pydantic_object=Workforce)
        prompt = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "additional_info"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt | self.model | pydantic_parser
        while True:
            try:
                output = chain.invoke({"prompt": self.prompt, "additional_info": self.additional_info})
                break
            except ValueError as e:
                print(f"ValueError occurred: {e}, retrying...")
        positions, roles, functions = output.positions, output.roles, output.functions
        for position, role, function in zip(positions, roles, functions):
            workforce.append(WorkerAgent(position, role, function, self.model))
        return workforce
        
    def _generate_response(self):
        sample = random.uniform(0, 1) # Bernoulli variable to prevent immediate termination

        # If the sample is less than the stopping probability, allow the director to decide whether to stop the conversation
        # Otherwise, continue the conversation
        if sample < self.stopping_probability:
            stop_prompt_template = """{prompt}
            
            Is the above prompt optimised enough to instruct a large language model?
            If yes, return the boolean value true. If no, return the boolean value false.

            Your output should be in a dictionary: {{"stop": true}} or {{"stop": false}}.

            {format_instructions}

            Do nothing else. 
            """
            pydantic_parser = PydanticOutputParser(pydantic_object=StopBoolean)
            stop_prompt = PromptTemplate(
                system_message=self.system_message,
                template=stop_prompt_template,
                input_variables=["prompt"],
                partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
            )
            chain = stop_prompt | self.model | pydantic_parser
            for _ in range(3):
                try:
                    output = chain.invoke({"prompt": self.prompt})
                    self.stop = output.stop
                    break
                except ValueError as e:
                    print(f"ValueError occurred: {e}, retrying...")
            else:
                print("Failed to generate the response. Defaulting to continuing the conversation")
                self.stop = False
        else:
            self.stop = False

        print(f"\tStop? {self.stop}\n")

        response_prompt_template = """{feedback}
        
        Think carefully about the above feedback and use it to make improvements to the current prompt: {prompt}.
        Do not remove any placeholder text in curly braces. This text is essential for the prompt to function correctly.
        Remember, the prompt should instruct a large language model to perform a task and improvements should be applied with that in mind.

        {termination_clause}

        Yor output should be in a dictionary: {{"prompt": "your prompt here"}}.

        {format_instructions}
        
        Do nothing else.
        """
        pydantic_parser = PydanticOutputParser(pydantic_object=Prompt)
        response_prompt = PromptTemplate(
            system_message=self.system_message,
            template=response_prompt_template,
            input_variables=["feedback", "prompt", "termination_clasue"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = response_prompt | self.model | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"feedback": self.recent_message, "prompt": self.prompt, "termination_clause": self.termination_clause if self.stop else self.continuation_clause})
                self.prompt = output.prompt
                return self.prompt
            except ValueError as e:
                print(f"ValueError occurred: {e}, retrying...")
        else:
            print("Failed to generate the response. Defaulting to the current prompt.")
            return self.prompt

    def _choose_next_worker(self) -> str:
        worker_positions = "\n".join(
            [f"{idx}: {position}" for idx, position in enumerate([worker.position for worker in self.workers])]
        )
        template ="""{prompt}
        
        Review the above prompt and the select the most appropriate next worker to provide feedback by choosing the index of the worker from the list below. 
        Do not select the worker who provided the most recent feedback: {previous_worker}.
        {worker_positions}

        Your output should be in a dictionary: {{"index": "your index here"}}.

        {format_instructions}

        Do nothing else.    
        """
        pydantic_parser = PydanticOutputParser(pydantic_object=WorkerIndex)
        prompt = PromptTemplate(
            system_message=self.system_message,
            template=template,
            input_variables=["prompt", "previous_worker", "worker_positions"],
            partial_variables={"format_instructions": pydantic_parser.get_format_instructions()},
        )
        chain = prompt | self.model | pydantic_parser
        for _ in range(3):
            try:
                output = chain.invoke({"prompt": self.prompt, "previous_worker": self.next_worker, "worker_positions": worker_positions})
                assert self.workers[output.index] != self.next_worker
                return output.index
            except ValueError as e:
                print(f"ValueError occurred: {e}, retrying...")
        else:
            output = 0
            print("Failed to select the next worker. Defaulting to the first worker.")
            return output

    def select_next_speaker(self):
        return self.chosen_worker_id

    def send(self) -> str:
        """
        Applies the feedback to the prompt and generates a new prompt for the next worker.
        """
        # Update prompt based on current discussion
        self.prompt = self._generate_response()

        if self.stop:
            message = self.prompt
        else:
            self.chosen_worker_id = self._choose_next_worker()
            self.next_worker = self.workers[self.chosen_worker_id].position
            print(f"\tNext worker: {self.next_worker}\n")
            message_template = """{next_worker}, please meticulously review and provide your feedback on the current prompt: {prompt}  
        Pay attention to any additional information provided: {additional_info}
        Be highly critical but keep your thoughts extremely concise and highly constructive. 
        Your response must not exceed 50 words. Do not forget to adhere to the word limit.

        Do nothing else.
        """
            message = message_template.format(prompt=self.prompt, next_worker=self.next_worker, additional_info=self.additional_info)

        return message

In [62]:
def select_next_speaker(step: int, agents: List[WorkerAgent], director: DirectorAgent) -> int:
    """
    If the step is even, then select the director
    Otherwise, the director selects the next speaker.
    """
    # the director speaks on odd steps
    if step % 2 == 1:
        idx = 0
    else:
        # here the director chooses the next speaker
        idx = director.select_next_speaker() + 1  # +1 because we excluded the director
    return idx

In [63]:
base_prompt = "Complete any Python function when provided with a function name and a docstring: {function_details}"
additional_info = "The task is related to programming. Do not remove any placeholders represented by curly braces."

director = DirectorAgent(
    base_prompt=base_prompt,
    model=ChatOpenAI(temperature=0.0),
    stopping_probability=0.5,
    additional_info=additional_info
    )

agents = [director]
for agent in director.workers:
    agents.append(agent)

simulator = OptimisationSimulator(agents=agents, selection_function=functools.partial(select_next_speaker, director=director))


In [64]:
for agent in agents:
    print(f"Position: {agent.position}, Role: {agent.role}, Function: {agent.function}\n")

Position: Director, Role: Oversee the optimisation of prompts, Function: Interact with workers to make improvements to optimise a prompt, where a prompt is defined as 'text input to an LLM that instructs the model to perform a task'.

Position: Python Developer, Role: Develop Python functions based on provided details, Function: Translate function details into Python code

Position: Software Engineer, Role: Implement and test Python functions, Function: Write efficient and effective Python code

Position: Data Analyst, Role: Analyze data related to Python functions, Function: Extract insights from data to optimize Python functions

Position: Quality Assurance Tester, Role: Ensure the quality and functionality of Python functions, Function: Identify and report any issues or bugs in Python functions



In [65]:
simulator.reset()
simulator.start()
print(f"(User): {base_prompt}")
print("\n")

while True:
    position, message = simulator.step()
    print(f"({position}): {message}")
    print("\n")
    if director.stop:
        break

(User): Complete any Python function when provided with a function name and a docstring: {function_details}


	Stop? False

	Next worker: Data Analyst

(Director): Data Analyst, please meticulously review and provide your feedback on the current prompt: Complete any Python function when provided with a function name and a docstring: {function_details}  
        Pay attention to any additional information provided: The task is related to programming. Do not remove any placeholders represented by curly braces.
        Be highly critical but keep your thoughts extremely concise and highly constructive. 
        Your response must not exceed 50 words. Do not forget to adhere to the word limit.

        Do nothing else.
        


(Data Analyst): The prompt lacks clarity on the specific requirements for completing the Python function. It needs to define the expected inputs, outputs, and constraints clearly. Additionally, it should specify the expected behavior of the function to ensure accu

In [67]:
print(director.prompt)

Complete a Python function that takes a function name and a docstring as input and returns the function details. The function should include the necessary inputs, outputs, and constraints in the docstring. The function signature should be: def get_function_details(function_name: str, docstring: str) -> dict:. Ensure that the docstring follows a specific format to extract function details effectively. Do not remove any placeholder text in curly braces. This text is essential for the prompt to function correctly.
